# Специализированные коннекторы GraphArchitect

Примеры использования сложных типов коннекторов из отчета:
- Музыкальный анализ
- МРТ изображения
- Графики физических процессов
- Картографические данные

In [1]:
import sys
from pathlib import Path

# Добавляем GraphArchitect
grapharchitect_path = Path.cwd().parent.parent / "src" / "GraphArchitectLib"
sys.path.insert(0, str(grapharchitect_path))

from grapharchitect.entities.connectors.specialized_connectors import SpecializedConnectorFactory
from grapharchitect.entities.connectors.connector import Connector
from grapharchitect.entities.base_tool import BaseTool
from grapharchitect.services.embedding.simple_embedding_service import SimpleEmbeddingService

print("✓ Компоненты импортированы")

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.


✓ Компоненты импортированы


## 1. Разбор специализированных коннекторов

Исследуем свойства сложных коннекторов.

In [2]:
factory = SpecializedConnectorFactory

# Получаем все коннекторы
connectors = factory.get_all_specialized_connectors()

print(f"Доступно специализированных коннекторов: {len(connectors)}\n")

for name, conn in connectors.items():
    print(f"=== {name} ===")
    print(f"Формат: {conn.format}")
    print("Свойства:")
    for key, value in conn.properties.items():
        print(f"  - {key}: {value}")
    print()

Доступно специализированных коннекторов: 12

=== song_analysis ===
Формат: text|report
Свойства:
  - language: russian
  - encoding: ['UTF-8', 'Windows-1251', 'KOI8-R']
  - length: 5000
  - style: simplified
  - knowledge_domain: general
  - template: structured_text
  - content_type: song_analysis

=== physics_graph ===
Формат: text|report
Свойства:
  - language: russian
  - encoding: ['UTF-8', 'Windows-1251', 'KOI8-R']
  - length_range: [100, 10000]
  - style: technical
  - knowledge_domain: physics
  - template: graph_analysis
  - content_type: physics_process

=== biology_reasoning ===
Формат: text|answer
Свойства:
  - language: russian
  - encoding: ['UTF-8', 'Windows-1251', 'KOI8-R']
  - length_range: [100, 10000]
  - style: scientific
  - format: dialog
  - knowledge_domain: biology
  - content_type: reasoning

=== math_reasoning ===
Формат: text|reasoning
Свойства:
  - language: russian
  - encoding: ['UTF-8', 'Windows-1251', 'KOI8-R']
  - length_range: [1000, 3000]
  - reasoni

## 2. Использование в инструментах

Создадим специализированные инструменты с этими коннекторами.

In [3]:
class MedicalAnalysisTool(BaseTool):
    """Анализатор медицинских изображений (МРТ)."""
    
    def __init__(self):
        super().__init__()
        self.metadata.tool_name = "MRI Analyzer"
        self.metadata.reputation = 0.95
        
        self.input = Connector("image", "raw")
        self.output = SpecializedConnectorFactory.create_mri_connector()
    
    def execute(self, input_data):
        return "[MRI Analyzer] Segments detected: 3 regions"


class SongAnalyzerTool(BaseTool):
    """Музыкальный аналитик."""
    
    def __init__(self):
        super().__init__()
        self.metadata.tool_name = "Song Deep Analyzer"
        self.metadata.reputation = 0.88
        
        self.input = Connector("audio", "song")
        self.output = SpecializedConnectorFactory.create_song_analysis_connector()
    
    def execute(self, input_data):
        return "[Song Analyzer] Detailed report on lyrics and composition"

# Проверяем
mri_tool = MedicalAnalysisTool()
song_tool = SongAnalyzerTool()

print(f"Tool: {mri_tool.metadata.tool_name}")
print(f"  Input: {mri_tool.input.format}")
print(f"  Output: {mri_tool.output.format}")
print(f"  Properties: {mri_tool.output.properties}")

print(f"\nTool: {song_tool.metadata.tool_name}")
print(f"  Input: {song_tool.input.format}")
print(f"  Output: {song_tool.output.format}")
print(f"  Properties: {song_tool.output.properties}")

Tool: MRI Analyzer
  Input: image|raw
  Output: image|reasoning
  Properties: {'color': 'RGB', 'pixel_type': ['uint8', 'uint16', 'float32'], 'resolutions': ['640x480', '1280x720', '1920x1080'], 'reasoning_type': 'hypothesis_verification', 'content': 'MRI', 'knowledge_domain': 'medicine', 'content_type': 'medical_imaging'}

Tool: Song Deep Analyzer
  Input: audio|song
  Output: text|report
  Properties: {'language': 'russian', 'encoding': ['UTF-8', 'Windows-1251', 'KOI8-R'], 'length': 5000, 'style': 'simplified', 'knowledge_domain': 'general', 'template': 'structured_text', 'content_type': 'song_analysis'}


## 3. Планирование с учетом специфики

Найдем инструменты для специфических задач.

In [4]:
from grapharchitect.services.graph_strategy_finder import GraphStrategyFinder
from grapharchitect.services.pathfinding_algorithm import PathfindingAlgorithm

# База инструментов
tools = [mri_tool, song_tool]

# Задача: Анализ МРТ
target_mri = SpecializedConnectorFactory.create_mri_connector()

finder = GraphStrategyFinder()

strategies = finder.find_strategies(
    tools=tools,
    start_format="image|raw",
    end_format=target_mri.format,
    algorithm=PathfindingAlgorithm.DIJKSTRA,
    limit=1
)

if strategies:
    print("Найдена стратегия для анализа МРТ:")
    print(f"  Tool: {strategies[0][0].metadata.tool_name}")
    
    # Проверка совместимости свойств
    output_props = strategies[0][0].output.properties
    print("\nСвойства результата:")
    print(f"  Domain: {output_props.get('knowledge_domain')}")
    print(f"  Resolution: {output_props.get('resolutions')}")
    print(f"  Color: {output_props.get('color')}")
else:
    print("Стратегия не найдена")

Найдена стратегия для анализа МРТ:
  Tool: MRI Analyzer

Свойства результата:
  Domain: medicine
  Resolution: ['640x480', '1280x720', '1920x1080']
  Color: RGB


## Итоги

Специализированные коннекторы позволяют:
1. Описывать сложные форматы данных (аудио, изображения, отчеты)
2. Указывать детальные свойства (битрейт, разрешение, стиль)
3. Строить точные цепочки обработки для специфических доменов

**Доступные типы**:
- Song Analysis
- Physics Graph
- Biology/Math Reasoning
- Audio Description
- Medical Imaging (MRI)
- Maps